In [1]:
!nvidia-smi

Wed Mar 30 04:11:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 461.79       Driver Version: 461.79       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 305... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   54C    P0    14W /  N/A |    310MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

In [3]:
# setup seed for experiment reproducibility
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(42)

In [4]:
# create node feature matrix and node-index mapping
with open("./data/features.txt") as file:
    features = file.readlines()
node2idx = {}
idx = 0
x = []
for feature in features:
    feature = feature.replace('\n', '').split(' ')
    node2idx[int(feature[0])] = idx
    x.append([int(f) for f in feature[1:]])
    idx += 1
idx2node = {idx: node for node, idx in node2idx.items()}
x = torch.tensor(x, dtype=torch.float)
num_node = x.size()[0]
num_feature = x.size()[1]

In [5]:
# create edge index
with open("./data/edges.txt") as file:
    edges = file.readlines()
edge_index1 = []
edge_index2 = [] 
for edge in edges:
    nodes = edge.replace('\n', '').split('\t')
    idx1 = node2idx[int(nodes[0])]
    idx2 = node2idx[int(nodes[1])]
    edge_index1.append(idx1)
    edge_index1.append(idx2)
    edge_index2.append(idx2)
    edge_index2.append(idx1)
edge_index = torch.tensor([edge_index1, edge_index2], dtype=torch.long)

In [6]:
# create class-index mapping
c2idx = {"Class_0": 0, "Class_1": 1, "Class_2": 2, "Class_3": 3, "Class_4": 4, "Class_5": 5, "Class_6": 6}
idx2c = {label: tag for tag, label in c2idx.items()}
num_class = len(c2idx)

# create masks and node-level targets
y = [0 for i in range(num_node)]
def loadData(path, test=False):
    df = pd.DataFrame(pd.read_csv(path))
    data = df["id"].tolist()
    mask = [False for i in range(num_node)]
    if (not test):
        label = [c2idx[i] for i in df["label"].tolist()]
    for i in range(len(data)):
        idx = node2idx[data[i]]
        mask[idx] = True
        if (not test):
            y[idx] = label[i]
    return mask
train_mask = torch.tensor(loadData("./data/train_labels.csv"))
val_mask = torch.tensor(loadData("./data/val_labels.csv"))
test_mask = torch.tensor(loadData("./data/test_idx.csv", True))
y = torch.tensor(y)

In [7]:
# get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# build dataset
dataset = Data(edge_index=edge_index, train_mask=train_mask, val_mask=val_mask, test_mask=test_mask, x=x, y=y).to(device)
print(dataset)

Using cuda device
Data(x=[2707, 1433], edge_index=[2, 10854], y=[2707], train_mask=[2707], val_mask=[2707], test_mask=[2707])


In [8]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(num_feature, 16)
        self.conv2 = GCNConv(16, num_class)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [9]:
# total training epochs
epochs = 50

# GCN model
model = GCN().to(device)

# Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# cosine learning rate decay
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

In [10]:
for t in range(epochs):
    print('\n', "=" * 15, "Epoch", t + 1, "=" * 15)
    
    # turn on training mode
    model.train()
    optimizer.zero_grad()
    
    # compute prediction error
    out = model(dataset)
    loss = F.nll_loss(out[dataset.train_mask], dataset.y[dataset.train_mask])
    
    # backpropagation
    loss.backward()
    
    # update optimizer and scheduler
    optimizer.step()
    scheduler.step()
    
    # turn on evaluation mode
    model.eval()
    
    # compute training loss
    pred = model(dataset).argmax(dim=1)
    correct = (pred[dataset.train_mask] == dataset.y[dataset.train_mask]).sum()
    acc = int(correct) / int(dataset.train_mask.sum())
    print(f'Train Accuracy: {acc:.4f}')
    
    # compute validation loss
    correct = (pred[dataset.val_mask] == dataset.y[dataset.val_mask]).sum()
    acc = int(correct) / int(dataset.val_mask.sum())
    print(f'Validation Accuracy: {acc:.4f}')


 =============== Epoch 1 ===============
Train Accuracy: 0.4800
Validation Accuracy: 0.3669

 =============== Epoch 2 ===============
Train Accuracy: 0.5200
Validation Accuracy: 0.4208

 =============== Epoch 3 ===============
Train Accuracy: 0.5750
Validation Accuracy: 0.4562

 =============== Epoch 4 ===============
Train Accuracy: 0.6350
Validation Accuracy: 0.4938

 =============== Epoch 5 ===============
Train Accuracy: 0.6800
Validation Accuracy: 0.5308

 =============== Epoch 6 ===============
Train Accuracy: 0.7150
Validation Accuracy: 0.5654

 =============== Epoch 7 ===============
Train Accuracy: 0.7500
Validation Accuracy: 0.6115

 =============== Epoch 8 ===============
Train Accuracy: 0.8150
Validation Accuracy: 0.6531

 =============== Epoch 9 ===============
Train Accuracy: 0.8950
Validation Accuracy: 0.7015

 =============== Epoch 10 ===============
Train Accuracy: 0.9050
Validation Accuracy: 0.7438

 =============== Epoch 11 ===============
Train Accuracy: 0.9150
Val

In [11]:
# save the model
path = "./aist4010-asm3.pth"
torch.save(model.state_dict(), path)

# output the result
test_data = pd.DataFrame(pd.read_csv("./data/test_idx.csv"))["id"].tolist()
node_pred = [idx2c[pred[dataset.test_mask][i].item()] for i in range(len(pred[dataset.test_mask]))]
output = {'id': np.array(test_data), 'label': np.array(node_pred)}
output_df = pd.DataFrame(output).set_index('id')
output_df.to_csv('output.csv')